In [ ]:
class SeqModule(nn.Module):
    def __init__(self, lat_size):
        super(SeqModule, self).__init__()
        self.lat_size = lat_size
        self.flatten = nn.Flatten()
        self.bigru = nn.GRU(
            input_size=data_file.num_features * data_file.num_lanes * 4,
            hidden_size=lat_size,
            num_layers=1,
            batch_first=True,
            bidirectional=True
        )

    def forward(self, inputs):
        inputs = inputs.view(inputs.size(0), data_file.num_veh, -1)
        print(inputs.size())
        # Create mask
        lengths = (inputs != 0).all(dim=2).sum(dim=1).cpu()
        print(lengths)  # Calculate sequence lengths
        packed_inputs = nn.utils.rnn.pack_padded_sequence(inputs, lengths, batch_first=True, enforce_sorted=False)
        packed_output, hidden = self.bigru(packed_inputs)
        output, _ = nn.utils.rnn.pad_packed_sequence(packed_output, batch_first=True)

        hidden_forward = hidden[-2, :, :]
        hidden_backward = hidden[-1, :, :]

        return output, (hidden_forward, hidden_backward)
